In [1]:
import lxml, bs4, requests
from bs4 import BeautifulSoup

In [2]:
def dutch_book_stakes(odds1, odds2, team1, team2, bet = 100):   
    #calculate first stake
    s1 = 1 - odds1/(odds1+odds2)
    s1 = round(s1, 4)*bet
    #calculate second stake
    s2 = bet - s1
    l=max(len(team1), len(team2))

    print(f"{' '*(l-4)} team | stake | payout")
    print("-" * 40)
    print(f" {team1:{l}} | {s1:5.5} | {s1*odds1:3.6}\n {team2:{l}} | {s2:5.5} | {s1*odds1:3.6}")
    
    
dutch_book_stakes(2.2, 3.3, "abc", "defgh")

  team | stake | payout
----------------------------------------
 abc   |  60.0 | 132.0
 defgh |  40.0 | 132.0


In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

In [4]:
import time
from operator import itemgetter

In [10]:
import pandas as pd

In [5]:
url = "https://www.hltv.org/matches/2329667/mousesports-vs-natus-vincere-esl-pro-league-season-8-finals"


In [12]:

def get_bets(url, driver):
    odds = []
    driver.get(url)
    bo = driver.find_elements_by_css_selector(".veto-box")[0].text[8]
    if bo == "2":
        return None
    
    
    allowed_providers = ["egb", "betway", "bet365", "1xbet", "22bet", "ggbet", "lootbet"]
    time.sleep(2)
    for betting_box in driver.find_elements_by_css_selector(".betting"):
        for line in betting_box.find_elements_by_css_selector(".betting_provider"):
            cell_entries = []
            for cell in line.find_elements_by_css_selector("td"):
                cell_entries.append(cell.text)
            if len(cell_entries) > 2 and cell_entries[2] == "-" and "%" not in cell_entries[-1]:
                provider = line.get_attribute("class").split(" ")[-2].split("_")[1]
                if provider in allowed_providers:
                    odds.append([provider, cell_entries[1], cell_entries[-1]])
    split_url = url[37:].split("-")
    if "vs" in split_url:
        ix = split_url.index("vs")
        team1 = " ".join(split_url[:ix])
        team2 = split_url[ix+1]
    else: 
        team1 = split_url[0]
        team2 = split_url[2]
    
    print(f"    provider | {team1} | {team2}")
    print("-"*13+"+"+"-"*(2+max(5,len(team1)))+"+"+"-"*(1+max(5,len(team2))))
    for odd in odds: 
        print(f'{odd[0]:{12}} | {" "*(len(team1)-5)}{odd[1]:{5}} | {" "*(len(team2)-5)}{odd[2]:{5}}')
    
    print("-"*13+"-"+"-"*(2+max(5,len(team1)))+"-"+"-"*(1+max(5,len(team2)))+"\n")
    return odds, team1, team2    

In [13]:
driver = webdriver.Chrome("chromedriver_win32/chromedriver.exe")
driver.get("https://www.hltv.org")
all_odds = []
matches = []
for match in driver.find_elements_by_css_selector(".hotmatch-box"):
    matches.append(match.get_attribute("href"))

time.sleep(1.337)
    
for match in matches:
    page_data = get_bets(match, driver)
    if page_data == None or len(page_data[0]) < 1:
        continue
    all_odds.append(page_data)
    odds, team1, team2 = all_odds[-1]
    df = pd.DataFrame(odds, columns=["provider", "team1", "team2"])
    df["team1"] = pd.to_numeric(df["team1"])
    df["team2"] = pd.to_numeric(df["team2"])
    t1_max = df["team1"].max()
    t2_max = df["team2"].max()
    print(f"best odds: {team1}: {t1_max}({df[df['team1']==t1_max].iloc[0,0]}) {team2}: {t2_max}({df[df['team2']==t2_max].iloc[0,0]})")
    dutch_book_stakes(t1_max, t2_max, team1, team2)
    print(2*"--------------------"+"\n")
driver.quit()

    provider | rog fl0m | rog
-------------+----------+------
egb          |    2.35  | 1.49 
ggbet        |    1.86  | 1.86 
1xbet        |    1.66  | 1.89 
-------------------------------

best odds: rog fl0m: 2.35(egb) rog: 1.89(1xbet)
     team | stake | payout
----------------------------------------
 rog fl0m | 44.58 | 104.763
 rog      | 55.42 | 104.763
----------------------------------------

    provider | rog spunj | rog
-------------+-----------+------
--------------------------------

    provider | rog pronax | rog
-------------+------------+------
---------------------------------

    provider | london | cex
-------------+--------+------
egb          |  1.05  | 5.53 
betway       |  1.08  | 6.50 
lootbet      |  1.14  | 5.20 
ggbet        |  1.08  | 6.44 
bet365       |  1.10  | 6.50 
1xbet        |  1.09  | 6.40 
-----------------------------

best odds: london: 1.14(lootbet) cex: 6.5(betway)
   team | stake | payout
----------------------------------------
 london | 8